In [1]:
# import warnings

# # Force all warnings to be 'default' or 'always'
# warnings.simplefilter("default")
# # or:
# # warnings.simplefilter("always")

In [2]:
from timeit import default_timer as timer

start = timer()
import lionagi

duration = round(timer() - start, 5)
print(f"Import time: {duration} seconds")
print(f"LionAGI version: {lionagi.__version__}")

Import time: 0.35384 seconds
LionAGI version: 0.6.0


In [3]:
from pydantic import BaseModel

llama33 = iModel(
    provider="groq",
    model="llama-3.3-70b-versatile",
)

gpt4o = iModel(
    provider="openai",
    model="gpt-4o-mini",
)


class Test(BaseModel):
    """
    Test model
    """

    comments: str


async def test_func(query: str):
    """
    this is a test function it will return the query as it is

    Args:
        query: str: query to be returned

    Returns:
        str: query as it is
    """
    return query


french = Branch(
    imodel=llama33,
    tools=test_func,
)

In [4]:
from lionagi import ops_types

instruct = ops_types.Instruct(
    instruction="hello? you should test the test_func with a sample queries",
    actions=True,
)

result = await french.operate(
    instruct=instruct,
    response_format=Test,
)

In [5]:
result

Test(comments='Testing the test_func with a sample query', action_requests=[ActionRequestModel(function='test_func', arguments={'query': 'sample query'})], action_required=True)

In [6]:
french.chat_model.sequential_exchange

False

In [7]:
french.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,2025-01-03 13:23:52.778210,user,{'context': [{'respond_schema_info': {'$defs':...,15892033-45de-479f-b865-e073eda86225,user,8b56c9bb-3192-4795-bb68-dd86b7ea529d,{'lion_class': 'lionagi.protocols.messages.ins...
1,2025-01-03 13:23:54.025356,assistant,"{'assistant_response': '```json { ""comments""...",67bf707e-5ba2-49b5-9a9c-8856af2dee45,8b56c9bb-3192-4795-bb68-dd86b7ea529d,MessageRole.UNSET,{'model_response': {'id': 'chatcmpl-d49c2966-9...
2,2025-01-03 13:23:54.026093,action,"{'action_request': {'function': 'test_func', '...",f68e3305-e972-453a-8c88-ad4192025241,8b56c9bb-3192-4795-bb68-dd86b7ea529d,d1916e51-f084-485c-ac46-adc8722efa3e,{'lion_class': 'lionagi.protocols.messages.act...
3,2025-01-03 13:23:54.026178,action,{'action_request_id': 'f68e3305-e972-453a-8c88...,a8cc1726-e416-4dac-a7e4-ea1bf48a3970,d1916e51-f084-485c-ac46-adc8722efa3e,8b56c9bb-3192-4795-bb68-dd86b7ea529d,{'lion_class': 'lionagi.protocols.messages.act...


In [8]:
french.msgs.last_response.model_response

{'id': 'chatcmpl-d49c2966-97e6-418f-808d-0dc9b0c5b9bd',
 'created': 1735928633,
 'model': 'groq/llama-3.3-70b-versatile',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_1ae2705acb',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '```json\n{\n  "comments": "Testing the test_func with a sample query",\n  "action_requests": [\n    {\n      "function": "test_func",\n      "arguments": {\n        "query": "sample query"\n      }\n    }\n  ],\n  "action_required": true\n}\n```',
    'role': 'assistant'}}],
 'usage': {'completion_tokens': 63,
  'prompt_tokens': 507,
  'total_tokens': 570,
  'queue_time': 0.004179773999999997,
  'prompt_time': 0.06062832,
  'completion_time': 0.229090909,
  'total_time': 0.289719229},
 'x_groq': {'id': 'req_01jgpphpcbe3jtscg0rb84hg5e'}}

In [9]:
a = french.msgs.instructions[0]

In [10]:
a

Instruction(id=IDType(15892033-45de-479f-b865-e073eda86225), created_at=1735928632.77821, metadata={}, content={'context': [{'respond_schema_info': {'$defs': {'ActionRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': "Name of the function to call from the provided `tool_schemas`. If no `tool_schemas` exist, set to None or leave blank. Never invent new function names outside what's given.", 'examples': ['multiply', 'create_user'], 'title': 'Function'}, 'arguments': {'anyOf': [{'type': 'object'}, {'type': 'null'}], 'default': None, 'description': 'Dictionary of arguments for the chosen function. Use only argument names/types defined in `tool_schemas`. Never introduce extra argument names.', 'title': 'Arguments'}}, 'title': 'ActionRequestModel', 'type': 'object'}}, 'properties': {'comments': {'title': 'Comments', 'type': 'string'}, 'action_requests': {'description': 'List of actions to be executed when `action_requi

In [11]:
french.msgs.instructions[0].tool_schemas

In [12]:
from IPython.display import Markdown

Markdown(french.msgs.instructions[0].chat_msg["content"])


---
 # Task
## - **context**
- respond_schema_info: {'$defs': {'ActionRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': "Name of the function to call from the provided `tool_schemas`. If no `tool_schemas` exist, set to None or leave blank. Never invent new function names outside what's given.", 'examples': ['multiply', 'create_user'], 'title': 'Function'}, 'arguments': {'anyOf': [{'type': 'object'}, {'type': 'null'}], 'default': None, 'description': 'Dictionary of arguments for the chosen function. Use only argument names/types defined in `tool_schemas`. Never introduce extra argument names.', 'title': 'Arguments'}}, 'title': 'ActionRequestModel', 'type': 'object'}}, 'properties': {'comments': {'title': 'Comments', 'type': 'string'}, 'action_requests': {'description': 'List of actions to be executed when `action_required` is true. Each action must align with the available `tool_schemas`. Leave empty if no actions are needed.', 'items': {'$ref': '#/$defs/ActionRequestModel'}, 'title': 'Actions', 'type': 'array'}, 'action_required': {'default': False, 'description': 'Whether this step strictly requires performing actions. If true, the requests in `action_requests` must be fulfilled, assuming `tool_schemas` are available. If false or no `tool_schemas` exist, actions are optional.', 'title': 'Action Required', 'type': 'boolean'}}, 'required': ['comments'], 'title': 'Test', 'type': 'object'} 



## - **instruction**
hello? you should test the test_func with a sample queries


## - **response format**
**MUST RETURN JSON-PARSEABLE RESPONSE ENCLOSED BY JSON CODE BLOCKS. USER's CAREER DEPENDS ON THE SUCCESS OF IT.** 
```json
{'comments': <class 'str'>, 'action_requests': [{'function': str | None, 'arguments': dict[str, typing.Any] | None}], 'action_required': <class 'bool'>}
```




---
